![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Public/2.Text_Preprocessing_with_SparkNLP_Annotators_Transformers.ipynb)

**Note** Read this article if you want to understand the basic concepts in Spark NLP.

[Introduction to Spark NLP: Foundations and Basic Components](https://towardsdatascience.com/introduction-to-spark-nlp-foundations-and-basic-components-part-i-c83b7629ed59)


## **0. Colab Setup**

In [ ]:
!pip install -q pyspark==3.3.0  spark-nlp==4.2.4

### **1. Annotators and Transformer Concepts**

In Spark NLP, all Annotators are either Estimators or Transformers as we see in Spark ML. An Estimator in Spark ML is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model. A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer that transforms a DataFrame with features into a DataFrame with predictions.
In Spark NLP, there are two types of annotators: AnnotatorApproach and AnnotatorModel
AnnotatorApproach extends Estimators from Spark ML, which are meant to be trained through fit(), and AnnotatorModel extends Transformers which are meant to transform data frames through transform().
Some of Spark NLP annotators have a Model suffix and some do not. The model suffix is explicitly stated when the annotator is the result of a training process. Some annotators, such as Tokenizer are transformers but do not contain the suffix Model since they are not trained, annotators. Model annotators have a pre-trained() on its static object, to retrieve the public pre-trained version of a model.
Long story short, if it trains on a DataFrame and produces a model, it’s an AnnotatorApproach; and if it transforms one DataFrame into another DataFrame through some models, it’s an AnnotatorModel (e.g. WordEmbeddingsModel) and it doesn’t take Model suffix if it doesn’t rely on a pre-trained annotator while transforming a DataFrame (e.g. Tokenizer).

By convention, there are three possible names:

**Approach** — Trainable annotator

**Model** — Trained annotator

**nothing** — Either a non-trainable annotator with pre-processing
step or shorthand for a model

So for example, Stemmer doesn’t say Approach nor Model, however, it is a Model. On the other hand, Tokenizer doesn’t say Approach nor Model, but it has a TokenizerModel(). Because it is not “training” anything, but it is doing some preprocessing before converting into a Model.
When in doubt, please refer to official documentation and API reference.
Even though we will do many hands-on practices in the following articles, let us give you a glimpse to let you understand the difference between AnnotatorApproach and AnnotatorModel.
As stated above, Tokenizer is an AnnotatorModel. So we need to call fit() and then transform().

Now let’s see how this can be done in Spark NLP using Annotators and Transformers. Assume that we have the following steps that need to be applied one by one on a data frame.

- Split text into sentences
- Tokenize
- Normalize
- Get word embeddings

**What’s actually happening under the hood?**

When we fit() on the pipeline with Spark data frame (df), its text column is fed into DocumentAssembler() transformer at first and then a new column “document” is created in Document type (AnnotatorType). As we mentioned before, this transformer is basically the initial entry point to Spark NLP for any Spark data frame. Then its document column is fed into SentenceDetector() (AnnotatorApproach) and the text is split into an array of sentences and a new column “sentences” in Document type is created. Then “sentences” column is fed into Tokenizer() (AnnotatorModel) and each sentence is tokenized and a new column “token” in Token type is created. And so on. 

In [2]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

spark

Spark NLP version:  4.2.4
Apache Spark version:  3.3.0


### **Create Spark Dataframe**

In [9]:
text = 'Peter Parker is a nice guy and lives in New York'

spark_df = spark.createDataFrame([[text]]).toDF("text")

spark_df.show(truncate=60)

+------------------------------------------------+
|                                            text|
+------------------------------------------------+
|Peter Parker is a nice guy and lives in New York|
+------------------------------------------------+



In [10]:
from pyspark.sql.types import StringType, IntegerType

# if you want to create a spark datafarme from a list of strings

text_list = ['Peter Parker is a nice guy and lives in New York.', 'Bruce Wayne is also a nice guy and lives in Gotham City.']

spark.createDataFrame(text_list, StringType()).toDF("text").show(truncate=60)

+--------------------------------------------------------+
|                                                    text|
+--------------------------------------------------------+
|       Peter Parker is a nice guy and lives in New York.|
|Bruce Wayne is also a nice guy and lives in Gotham City.|
+--------------------------------------------------------+



In [11]:
from pyspark.sql import Row

spark.createDataFrame(list(map(lambda x: Row(text=x), text_list))).show(truncate=60)

+--------------------------------------------------------+
|                                                    text|
+--------------------------------------------------------+
|       Peter Parker is a nice guy and lives in New York.|
|Bruce Wayne is also a nice guy and lives in Gotham City.|
+--------------------------------------------------------+



In [12]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/jupyter/annotation/english/spark-nlp-basics/sample-sentences-en.txt

In [13]:
with open('./sample-sentences-en.txt') as f:
  print (f.read())

Peter is a very good person.
My life in Russia is very interesting.
John and Peter are brothers. However they don't support each other that much.
Lucas Nogal Dunbercker is no longer happy. He has a good car though.
Europe is very culture rich. There are huge churches! and big houses!


In [14]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [15]:
spark_df.select('text').show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [16]:
textFiles = spark.sparkContext.wholeTextFiles("./*.txt",4)
    
spark_df_folder = textFiles.toDF(schema=['path','text'])

spark_df_folder.show(truncate=30)

+------------------------------+------------------------------+
|                          path|                          text|
+------------------------------+------------------------------+
|file:/content/sample-senten...|Peter is a very good person...|
+------------------------------+------------------------------+



In [17]:
spark_df_folder.select('text').take(1)

[Row(text="Peter is a very good person.\nMy life in Russia is very interesting.\nJohn and Peter are brothers. However they don't support each other that much.\nLucas Nogal Dunbercker is no longer happy. He has a good car though.\nEurope is very culture rich. There are huge churches! and big houses!")]

In [18]:
spark_df_folder.select('text').collect()

[Row(text="Peter is a very good person.\nMy life in Russia is very interesting.\nJohn and Peter are brothers. However they don't support each other that much.\nLucas Nogal Dunbercker is no longer happy. He has a good car though.\nEurope is very culture rich. There are huge churches! and big houses!")]

### **Transformers**

What are we going to do if our DataFrame doesn’t have columns in those type? Here comes transformers. In Spark NLP, we have five different transformers that are mainly used for getting the data in or transform the data from one AnnotatorType to another.


Each annotator accepts certain types of columns and outputs new columns in another type (we call this AnnotatorType).

In Spark NLP, we have the following types: 

>`Document`, `token`, `chunk`, `pos`, `word_embeddings`, `date`, `entity`, `sentiment`, `named_entity`, `dependency`, `labeled_dependency`. 

That is, the DataFrame you have needs to have a column from one of these types if that column will be fed into an annotator; otherwise, you’d need to use one of the Spark NLP transformers.

##  **Document Assembler**

In Spark NLP, we have five different transformers that are mainly used for getting the data in or transform the data from one AnnotatorType to another.

That is, the DataFrame you have needs to have a column from one of these types if that column will be fed into an annotator; otherwise, you’d need to use one of the Spark NLP transformers. Here is the list of transformers: DocumentAssembler, TokenAssembler, Doc2Chunk, Chunk2Doc, and the Finisher.

So, let’s start with DocumentAssembler(), an entry point to Spark NLP annotators.

To get through the process in Spark NLP, we need to get raw data transformed into Document type at first. 

DocumentAssembler() is a special transformer that does this for us; it creates the first annotation of type Document which may be used by annotators down the road.

DocumentAssembler() comes from sparknlp.base class and has the following settable parameters. 
  ```

In [19]:
spark_df.show()

+--------------------+
|                text|
+--------------------+
|Peter is a very g...|
|My life in Russia...|
|John and Peter ar...|
|Lucas Nogal Dunbe...|
|Europe is very cu...|
+--------------------+



In [20]:
spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [21]:
from sparknlp.base import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")\
    .setCleanupMode("shrink")

doc_df = documentAssembler.transform(spark_df)

doc_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Peter is a very good person.                                                 |[{document, 0, 27, Peter is a very good person., {sentence -> 0}, []}]                                                 |
|My life in Russia is very interesting.                                       |[{document, 0, 37, My life in Russia is very interesting., {sentence -> 0}, []}]                                       |


At first, we define DocumentAssembler with desired parameters and then transform the data frame with it. The most important point to pay attention to here is that you need to use a String or String[Array] type column in .setInputCol(). So it doesn’t have to be named as text. You just use the column name as it is.

In [22]:
doc_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)



In [23]:
doc_df.select('document.result','document.begin','document.end').show(truncate=False)

+-------------------------------------------------------------------------------+-----+----+
|result                                                                         |begin|end |
+-------------------------------------------------------------------------------+-----+----+
|[Peter is a very good person.]                                                 |[0]  |[27]|
|[My life in Russia is very interesting.]                                       |[0]  |[37]|
|[John and Peter are brothers. However they don't support each other that much.]|[0]  |[76]|
|[Lucas Nogal Dunbercker is no longer happy. He has a good car though.]         |[0]  |[67]|
|[Europe is very culture rich. There are huge churches! and big houses!]        |[0]  |[68]|
+-------------------------------------------------------------------------------+-----+----+



The new column is in an array of struct type and has the parameters shown above. The annotators and transformers all come with universal metadata that would be filled down the road depending on the annotators being used. Unless you want to append other Spark NLP annotators to DocumentAssembler(), you don’t need to know what all these parameters mean for now. So we will talk about them in the following articles. You can access all these parameters with {column name}.{parameter name}.

Let’s print out the first item’s result.

In [24]:
doc_df.select("document.result").take(1)

[Row(result=['Peter is a very good person.'])]

If we would like to flatten the document column, we can do as follows.


In [25]:
import pyspark.sql.functions as F

doc_df.withColumn(
    "tmp", 
    F.explode("document"))\
    .select("tmp.*")\
    .show(truncate=False)

+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|annotatorType|begin|end|result                                                                       |metadata       |embeddings|
+-------------+-----+---+-----------------------------------------------------------------------------+---------------+----------+
|document     |0    |27 |Peter is a very good person.                                                 |{sentence -> 0}|[]        |
|document     |0    |37 |My life in Russia is very interesting.                                       |{sentence -> 0}|[]        |
|document     |0    |76 |John and Peter are brothers. However they don't support each other that much.|{sentence -> 0}|[]        |
|document     |0    |67 |Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |{sentence -> 0}|[]        |
|document     |0    |68 |Europe is very culture rich. There are huge churches! and 

## **Sentence Detector**

Finds sentence bounds in raw text. 

In [26]:
from sparknlp.annotator import *

# we feed the document column coming from Document Assembler

sentenceDetector = SentenceDetector()\
      .setInputCols(['document'])\
      .setOutputCol('sentences')

In [27]:
sentenceDetector.extractParamMap()

{Param(parent='SentenceDetector_879de529b3e0', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='SentenceDetector_879de529b3e0', name='useAbbreviations', doc='whether to apply abbreviations at sentence detection'): True,
 Param(parent='SentenceDetector_879de529b3e0', name='detectLists', doc='whether detect lists during sentence detection'): True,
 Param(parent='SentenceDetector_879de529b3e0', name='useCustomBoundsOnly', doc='Only utilize custom bounds in sentence detection'): False,
 Param(parent='SentenceDetector_879de529b3e0', name='customBounds', doc='characters used to explicitly mark sentence bounds'): [],
 Param(parent='SentenceDetector_879de529b3e0', name='customBoundsStrategy', doc='How to return matched custom bounds'): 'none',
 Param(parent='SentenceDetector_879de529b3e0', name='explodeSentences', doc='whether to explode each sentence into a different row, for better parallelization. Defaults to false.'): False,

In [28]:
doc_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+
|Peter is a very good person.                                                 |[{document, 0, 27, Peter is a very good person., {sentence -> 0}, []}]                                                 |
|My life in Russia is very interesting.                                       |[{document, 0, 37, My life in Russia is very interesting., {sentence -> 0}, []}]                                       |


In [29]:
sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                         |document                                                                                                               |sentences                                                                                                                                                                                          |
+-----------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------+---------

In [30]:
sent_df.select('sentences').take(3)

[Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='Peter is a very good person.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=37, result='My life in Russia is very interesting.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(sentences=[Row(annotatorType='document', begin=0, end=27, result='John and Peter are brothers.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='document', begin=29, end=76, result="However they don't support each other that much.", metadata={'sentence': '1'}, embeddings=[])])]

In [31]:
sent_df.select('sentences.result').take(3)

[Row(result=['Peter is a very good person.']),
 Row(result=['My life in Russia is very interesting.']),
 Row(result=['John and Peter are brothers.', "However they don't support each other that much."])]

In [32]:
sent_df.select('sentences.metadata').take(3)

[Row(metadata=[{'sentence': '0'}]),
 Row(metadata=[{'sentence': '0'}]),
 Row(metadata=[{'sentence': '0'}, {'sentence': '1'}])]

In [33]:
text ='The patient was prescribed 1 capsule of Advil for 5 days. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months.'
text


'The patient was prescribed 1 capsule of Advil for 5 days. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months.'

In [34]:
spark_df = spark.createDataFrame([[text]]).toDF("text")

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                                                                                                                                                                                                                                           |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [35]:
spark_df.show(truncate=50)

+--------------------------------------------------+
|                                              text|
+--------------------------------------------------+
|The patient was prescribed 1 capsule of Advil f...|
+--------------------------------------------------+



In [36]:
doc_df = documentAssembler.transform(spark_df)

sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                          document|                                         sentences|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|The patient was prescribed 1 capsule of Advil f...|[{document, 0, 334, The patient was prescribed ...|[{document, 0, 56, The patient was prescribed 1...|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+



In [37]:
sent_df.select('sentences.result').take(1)

[Row(result=['The patient was prescribed 1 capsule of Advil for 5 days.', 'He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.', 'It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months.'])]

In [38]:
# setExplodeSentences: Whether to split sentences into different Dataset rows. Useful for higher parallelism in fat rows. Defaults to false.

sentenceDetector.setExplodeSentences(True)

SentenceDetector_879de529b3e0

In [39]:
sent_df = sentenceDetector.transform(doc_df)

sent_df.show(truncate=50)

+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|                                              text|                                          document|                                         sentences|
+--------------------------------------------------+--------------------------------------------------+--------------------------------------------------+
|The patient was prescribed 1 capsule of Advil f...|[{document, 0, 334, The patient was prescribed ...|[{document, 0, 56, The patient was prescribed 1...|
|The patient was prescribed 1 capsule of Advil f...|[{document, 0, 334, The patient was prescribed ...|[{document, 58, 240, He was seen by the endocri...|
|The patient was prescribed 1 capsule of Advil f...|[{document, 0, 334, The patient was prescribed ...|[{document, 242, 334, It was determined that al...|
+--------------------------------------------------+------------------

In [40]:
sent_df.select('sentences.result').show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                   |
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[The patient was prescribed 1 capsule of Advil for 5 days.]                                                                                                                              |
|[He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.]|
|[It was determined that all SGLT2 inhibitors should be disc

In [41]:
from pyspark.sql import functions as F

sent_df.select(F.explode('sentences.result')).show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|The patient was prescribed 1 capsule of Advil for 5 days.                                                                                                                              |
|He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.|
|It was determined that all SGLT2 inhibitors should be discontinued in

**`.setCustomBounds([r"\\.", ";"])`**

**`.setCustomBoundsStrategy("append")`**


In [42]:
text = [
    ["Peter is a very good person."],
    ["My life in Russia is very interesting."], 
    ["John and Peter are brothers. However; they don't support each other that much."],
    ["Lucas Nogal Dunbercker is no longer happy. He has a good car though."],
    ["Europe is very culture rich. There are huge churches! and big houses!"]
    ]

In [43]:
spark_df = spark.createDataFrame(text).toDF("text")
spark_df.show(truncate=False)

+------------------------------------------------------------------------------+
|text                                                                          |
+------------------------------------------------------------------------------+
|Peter is a very good person.                                                  |
|My life in Russia is very interesting.                                        |
|John and Peter are brothers. However; they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.          |
|Europe is very culture rich. There are huge churches! and big houses!         |
+------------------------------------------------------------------------------+



In [44]:
doc_df = documentAssembler.transform(spark_df)

In [45]:
sent_df.select('sentences.result').take(5)

[Row(result=['The patient was prescribed 1 capsule of Advil for 5 days.']),
 Row(result=['He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day.']),
 Row(result=['It was determined that all SGLT2 inhibitors should be discontinued indefinitely fro 3 months.'])]

**`.setCustomBoundsStrategy("prepend")`**

In [46]:
sentenceDetector = SentenceDetector()\
      .setInputCols(['document'])\
      .setOutputCol('sentences')\
      .setCustomBounds([r"\.", ";", "!"])\
      .setCustomBoundsStrategy("prepend")

sent_df = sentenceDetector.transform(doc_df)
sent_df.show(truncate=False)

+------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                          |document                                                                                                                |sentences                                                                                                                                                                                                                                                              

In [47]:
sent_df.select('sentences.result').take(5)

[Row(result=['Peter is a very good person', '.']),
 Row(result=['My life in Russia is very interesting', '.']),
 Row(result=['John and Peter are brothers', '.', 'However', ';', "they don't support each other that much", '.']),
 Row(result=['Lucas Nogal Dunbercker is no longer happy', '.', 'He has a good car though', '.']),
 Row(result=['Europe is very culture rich', '.', 'There are huge churches', '!', 'and big houses', '!'])]

The separation of the sentences is determined according to the characters we set with custom bound. When we use `append`, sentences are differentiated according to the characters, if `prepend` is used, it also determines the characters as separate sentences.

### **Sentence Detector DL**

In [48]:
text ='The patient was prescribed 1 capsule of Advil for 5 days. He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and metformin 1000 mg two times a day. It was determined that all SGLT2 inhibitors should be discontinued indefinitely from 3 months.'

spark_df = spark.createDataFrame([[text]]).toDF("text")

doc_df = documentAssembler.transform(spark_df)

In [49]:
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")\

sent_dl_df = sentencerDL.transform(doc_df)

sent_dl_df.select(F.explode('sentences.result')).show(truncate=False)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|col                                                                                                                                                                                    |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|The patient was prescribed 1 capsule of Advil for 5 days.                                                                                                                              |
|He was seen by the endocrinology service and she was discharged on 40 units of insulin glargine at night, 12 units of insulin lispro with meals, and

In [50]:
documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentences')\
    
sentencerDL = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl", "en") \
    .setInputCols(["document"]) \
    .setOutputCol("sentences")


sd_pipeline = PipelineModel(stages=[documenter, sentenceDetector])
sd_model = LightPipeline(sd_pipeline)


# DL version
sd_dl_pipeline = PipelineModel(stages=[documenter, sentencerDL])
sd_dl_model = LightPipeline(sd_dl_pipeline)

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]


In [51]:
text = """John loves Mary.Mary loves Peter
Peter loves Helen .Helen loves John; 
Total: four people involved."""

# sd_model
for anno in sd_model.fullAnnotate(text)[0]["sentences"]:
    print("{}\t{}\t{}\t{}".format(
        anno.metadata["sentence"], anno.begin, anno.end, anno.result))

0	0	51	John loves Mary.Mary loves Peter
Peter loves Helen .
1	52	68	Helen loves John;
2	71	98	Total: four people involved.


In [52]:
# sd_dl_model
for anno in sd_dl_model.fullAnnotate(text)[0]["sentences"]:
    print("{}\t{}\t{}\t{}".format(
        anno.metadata["sentence"], anno.begin, anno.end, anno.result))

0	0	15	John loves Mary.
1	16	31	Mary loves Peter
2	33	51	Peter loves Helen .
3	52	68	Helen loves John;
4	71	98	Total: four people involved.


## Tokenizer

Identifies tokens with tokenization open standards. It is an **Annotator Approach, so it requires .fit()**.

A few rules will help customizing it if defaults do not fit user needs.

setExceptions(StringArray): List of tokens to not alter at all. Allows composite tokens like two worded tokens that the user may not want to split.





In [53]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

In [54]:
tokenizer.extractParamMap()

{Param(parent='Tokenizer_c81d2206ec09', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='Tokenizer_c81d2206ec09', name='targetPattern', doc='pattern to grab from text as token candidates. Defaults \\S+'): '\\S+',
 Param(parent='Tokenizer_c81d2206ec09', name='contextChars', doc='character list used to separate from token boundaries'): ['.',
  ',',
  ';',
  ':',
  '!',
  '?',
  '*',
  '-',
  '(',
  ')',
  '"',
  "'"],
 Param(parent='Tokenizer_c81d2206ec09', name='caseSensitiveExceptions', doc='Whether to care for case sensitiveness in exceptions'): True,
 Param(parent='Tokenizer_c81d2206ec09', name='minLength', doc='Set the minimum allowed length for each token'): 0,
 Param(parent='Tokenizer_c81d2206ec09', name='maxLength', doc='Set the maximum allowed length for each token'): 99999,
 Param(parent='Tokenizer_c81d2206ec09', name='inputCols', doc='previous annotations columns, if renamed'): ['document'],
 Param(parent='Token

In [55]:
text = 'Peter Parker (Spiderman) is a nice guy and lives in New York but has no e-mail!'

spark_df = spark.createDataFrame([[text]]).toDF("text")

In [56]:
doc_df = documentAssembler.transform(spark_df)

token_df = tokenizer.fit(doc_df).transform(doc_df)

token_df.show(truncate=100)

+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                           text|                                                                                            document|                                                                                               token|
+-------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|Peter Parker (Spiderman) is a nice guy and lives in New York but has no e-mail!|[{document, 0, 78, Peter Parker (Spiderman) is a nice guy and lives

In [57]:
token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(', 'Spiderman', ')', 'is', 'a', 'nice', 'guy', 'and', 'lives', 'in', 'New', 'York', 'but', 'has', 'no', 'e-mail', '!'])]

In [58]:
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token") \
    .setSplitChars(['-']) \
    .setContextChars(['?', '!']) \
    .addException("New York")

In [59]:
token_df = tokenizer.fit(doc_df).transform(doc_df)

token_df.select('token.result').take(1)

[Row(result=['Peter', 'Parker', '(Spiderman)', 'is', 'a', 'nice', 'guy', 'and', 'lives', 'in', 'New York', 'but', 'has', 'no', 'e', 'mail', '!'])]

## Regex Tokenizer

In [60]:
from pyspark.sql.types import StringType

content = "1. T1-T2 DATE**[12/24/13] $1.99 () (10/12), ph+ 90%"
pattern = "\\s+|(?=[-.:;*+,$&%\\[\\]])|(?<=[-.:;*+,$&%\\[\\]])"

df = spark.createDataFrame([content], StringType()).withColumnRenamed("value", "text")

documenter = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

regexTokenizer = RegexTokenizer() \
    .setInputCols(["sentence"]) \
    .setOutputCol("regexToken") \
    .setPattern(pattern) \
    .setPositionalMask(False)

docPatternRemoverPipeline = Pipeline().setStages([documenter,
                                                  sentenceDetector,
                                                  regexTokenizer])

result = docPatternRemoverPipeline.fit(df).transform(df)
result.show(truncate=False)

+---------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [61]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(result.regexToken.result).alias('regexToken')).toPandas()
result_df

,regexToken
0,1
1,.
2,T1
3,-
4,T2
5,DATE
6,*
7,*
8,[
9,12/24/13


## Stacking Spark NLP Annotators in Spark ML Pipeline

Spark NLP provides an easy API to integrate with Spark ML Pipelines and all the Spark NLP annotators and transformers can be used within Spark ML Pipelines. So, it’s better to explain Pipeline concept through Spark ML official documentation.

What is a Pipeline anyway? In machine learning, it is common to run a sequence of algorithms to process and learn from data. 

Apache Spark ML represents such a workflow as a Pipeline, which consists of a sequence of PipelineStages (Transformers and Estimators) to be run in a specific order.

In simple terms, a pipeline chains multiple Transformers and Estimators together to specify an ML workflow. We use Pipeline to chain multiple Transformers and Estimators together to specify our machine learning workflow.

A Pipeline is specified as a sequence of stages, and each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage. That is, the data are passed through the fitted pipeline in order. Each stage’s transform() method updates the dataset and passes it to the next stage. With the help of Pipelines, we can ensure that training and test data go through identical feature processing steps.

Now let’s see how this can be done in Spark NLP using Annotators and Transformers. Assume that we have the following steps that need to be applied one by one on a data frame.

- Split text into sentences
- Tokenize

And here is how we code this pipeline up in Spark NLP.

In [62]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               sentenceDetector,
                               tokenizer])

spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

pipelineModel = nlpPipeline.fit(spark_df)

In [63]:
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

spark_df.show(truncate=False)

+-----------------------------------------------------------------------------+
|text                                                                         |
+-----------------------------------------------------------------------------+
|Peter is a very good person.                                                 |
|My life in Russia is very interesting.                                       |
|John and Peter are brothers. However they don't support each other that much.|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |
|Europe is very culture rich. There are huge churches! and big houses!        |
+-----------------------------------------------------------------------------+



In [64]:
result = pipelineModel.transform(spark_df)

In [65]:
result.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                               sentences|                                   token|
+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|            Peter is a very good person.|[{document, 0, 27, Peter is a very go...|[{document, 0, 27, Peter is a very go...|[{token, 0, 4, Peter, {sentence -> 0}...|
|  My life in Russia is very interesting.|[{document, 0, 37, My life in Russia ...|[{document, 0, 37, My life in Russia ...|[{token, 0, 1, My, {sentence -> 0}, [...|
|John and Peter are brothers. However ...|[{document, 0, 76, John and Peter are...|[{document, 0, 27, John and Peter are...|[{token, 0, 3, John, {sentence -> 0},...|
|Luc

In [66]:
result.printSchema()

root
 |-- text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- sentences: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = tru

In [67]:
result.select('sentences.result').take(3)

[Row(result=['Peter is a very good person.']),
 Row(result=['My life in Russia is very interesting.']),
 Row(result=['John and Peter are brothers.', "However they don't support each other that much."])]

In [68]:
result.select('token').take(3)[2]

Row(token=[Row(annotatorType='token', begin=0, end=3, result='John', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=5, end=7, result='and', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=13, result='Peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=15, end=17, result='are', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=19, end=26, result='brothers', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=27, end=27, result='.', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=29, end=35, result='However', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=37, end=40, result='they', metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=42, end=46, result="don't", metadata={'sentence': '1'}, embeddings=[]), Row(annotatorType='token', begin=48, end=54, result='s

## Normalizer

Removes all dirty characters from text following a regex pattern and transforms words based on a provided dictionary

`setCleanupPatterns(patterns)`: Regular expressions list for normalization, defaults [^A-Za-z]

`setLowercase(value)`: lowercase tokens, default false

`setSlangDictionary(path)`: txt file with delimited words to be transformed into something else


In [69]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [70]:
from sparknlp.base import *
from sparknlp.annotator import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
    
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True)\
    .setCleanupPatterns(["[^\w\d\s]"]) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               normalizer])

result = nlpPipeline.fit(spark_df).transform(spark_df)

In [71]:
nlpPipeline.fit(spark_df).stages

[DocumentAssembler_1b51aeadfb9e,
 REGEX_TOKENIZER_00ac48bb8a5d,
 NORMALIZER_bfeabe2044f7]

In [72]:
result.select('token').take(2)

[Row(token=[Row(annotatorType='token', begin=0, end=4, result='Peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=6, end=7, result='is', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=9, result='a', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=11, end=14, result='very', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=16, end=19, result='good', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=21, end=26, result='person', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=27, end=27, result='.', metadata={'sentence': '0'}, embeddings=[])]),
 Row(token=[Row(annotatorType='token', begin=0, end=1, result='My', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=3, end=6, result='life', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=8, end=9, result='in',

In [73]:
result.select('normalized.result').take(2)

[Row(result=['peter', 'is', 'a', 'very', 'good', 'person']),
 Row(result=['my', 'life', 'in', 'russia', 'is', 'very', 'interesting'])]

In [74]:
result.select('normalized').take(2)

[Row(normalized=[Row(annotatorType='token', begin=0, end=4, result='peter', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=6, end=7, result='is', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=9, end=9, result='a', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=11, end=14, result='very', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=16, end=19, result='good', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=21, end=26, result='person', metadata={'sentence': '0'}, embeddings=[])]),
 Row(normalized=[Row(annotatorType='token', begin=0, end=1, result='my', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=3, end=6, result='life', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=8, end=9, result='in', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=11, end=16, r

## Document Normalizer

The DocumentNormalizer is an annotator that can be used after the DocumentAssembler to narmalize documents once that they have been processed and indexed .
It takes in input annotated documents of type Array AnnotatorType.DOCUMENT and gives as output annotated document of type AnnotatorType.DOCUMENT .




In [75]:
text = '''
  <div id="theworldsgreatest" class='my-right my-hide-small my-wide toptext' style="font-family:'Segoe UI',Arial,sans-serif">
    THE WORLD'S LARGEST WEB DEVELOPER SITE
    <h1 style="font-size:300%;">THE WORLD'S LARGEST WEB DEVELOPER SITE</h1>
    <p style="font-size:160%;">Lorem Ipsum is simply dummy text of the printing and typesetting industry. Lorem Ipsum has been the industry's standard dummy text ever since the 1500s, when an unknown printer took a galley of type and scrambled it to make a type specimen book. It has survived not only five centuries, but also the leap into electronic typesetting, remaining essentially unchanged. It was popularised in the 1960s with the release of Letraset sheets containing Lorem Ipsum passages, and more recently with desktop publishing software like Aldus PageMaker including versions of Lorem Ipsum..</p>
  </div>

</div>'''

In [76]:
spark_df = spark.createDataFrame([[text]]).toDF("text")

spark_df.show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|text                                                                                                                

In [77]:
documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument")

documentNormalizer.extractParamMap()

{Param(parent='DocumentNormalizer_d77593b015b9', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='DocumentNormalizer_d77593b015b9', name='action', doc='action to perform applying regex patterns on text'): 'clean',
 Param(parent='DocumentNormalizer_d77593b015b9', name='patterns', doc='normalization regex patterns which match will be removed from document. Defaults is <[^>]*>'): ['<[^>]*>'],
 Param(parent='DocumentNormalizer_d77593b015b9', name='replacement', doc='replacement string to apply when regexes match'): ' ',
 Param(parent='DocumentNormalizer_d77593b015b9', name='lowercase', doc='whether to convert strings to lowercase'): False,
 Param(parent='DocumentNormalizer_d77593b015b9', name='policy', doc='policy to remove pattern from text'): 'pretty_all',
 Param(parent='DocumentNormalizer_d77593b015b9', name='encoding', doc='file encoding to apply on normalized documents'): 'UTF-8',
 Param(parent='DocumentNormalizer_d7759

In [78]:
documentAssembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

#default
cleanUpPatterns = ["<[^>]*>"]

documentNormalizer = DocumentNormalizer() \
    .setInputCols("document") \
    .setOutputCol("normalizedDocument") \
    .setAction("clean") \
    .setPatterns(cleanUpPatterns) \
    .setReplacement(" ") \
    .setPolicy("pretty_all") \
    .setLowercase(True)

docPatternRemoverPipeline = Pipeline() \
    .setStages([documentAssembler,
                documentNormalizer])
    
pipelineModel = docPatternRemoverPipeline.fit(spark_df).transform(spark_df)

In [79]:
pipelineModel.select('normalizedDocument.result').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|result                                                                                                                                                                                                                                                                                                                                              

 for more examples : https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/annotation/english/document-normalizer/document_normalizer_notebook.ipynb

## Stopwords Cleaner

This annotator excludes from a sequence of strings (e.g. the output of a Tokenizer, Normalizer, Lemmatizer, and Stemmer) and drops all the stop words from the input sequences.




In [80]:
stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("token")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)\
    #.setStopWords(["no", "without"]) (e.g. read a list of words from a txt)

In [ ]:
stopwords_cleaner.getStopWords()

In [81]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               stopwords_cleaner])
 
spark_df = spark.read.text('./sample-sentences-en.txt').toDF('text')

result = nlpPipeline.fit(spark_df).transform(spark_df)

result.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                                   token|                             cleanTokens|
+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|            Peter is a very good person.|[{document, 0, 27, Peter is a very go...|[{token, 0, 4, Peter, {sentence -> 0}...|[{token, 0, 4, Peter, {sentence -> 0}...|
|  My life in Russia is very interesting.|[{document, 0, 37, My life in Russia ...|[{token, 0, 1, My, {sentence -> 0}, [...|[{token, 3, 6, life, {sentence -> 0},...|
|John and Peter are brothers. However ...|[{document, 0, 76, John and Peter are...|[{token, 0, 3, John, {sentence -> 0},...|[{token, 0, 3, John, {sentence -> 0},...|
|Luc

In [82]:
result.select('cleanTokens.result').take(1)

[Row(result=['Peter', 'good', 'person', '.'])]

## Token Assembler

In [83]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentenceDetector = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(False)\

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols("normalized")\
    .setOutputCol("cleanTokens")\
    .setCaseSensitive(False)\

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "cleanTokens"]) \
    .setOutputCol("clean_text")


nlpPipeline = Pipeline(stages=[documentAssembler,
                               sentenceDetector,
                               tokenizer,
                               normalizer,
                               stopwords_cleaner,
                               tokenassembler])

result = nlpPipeline.fit(spark_df).transform(spark_df)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|          normalized|         cleanTokens|          clean_text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[{document, 0, 27...|[{document, 0, 27...|[{token, 0, 4, Pe...|[{token, 0, 4, Pe...|[{token, 0, 4, Pe...|[{document, 0, 16...|
|My life in Russia...|[{document, 0, 37...|[{document, 0, 37...|[{token, 0, 1, My...|[{token, 0, 1, My...|[{token, 3, 6, li...|[{document, 0, 22...|
|John and Peter ar...|[{document, 0, 76...|[{document, 0, 27...|[{token, 0, 3, Jo...|[{token, 0, 3, Jo...|[{token, 0, 3, Jo...|[{document, 0, 18...|
|Lucas Nogal Dunbe...|[{document, 0, 67...|[{document, 0, 41...|[{token, 0, 4, Lu...|[{token, 0, 4, Lu...|

In [84]:
# if we use TokenAssembler().setPreservePosition(True), the original borders will be preserved (dropped & unwanted chars will be replaced by spaces)

result.select('clean_text').take(1)

[Row(clean_text=[Row(annotatorType='document', begin=0, end=16, result='Peter good person', metadata={'sentence': '0'}, embeddings=[])])]

In [85]:
result.select('text', F.explode(result.clean_text.result).alias('clean_text')).show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------+
|text                                                                         |clean_text                         |
+-----------------------------------------------------------------------------+-----------------------------------+
|Peter is a very good person.                                                 |Peter good person                  |
|My life in Russia is very interesting.                                       |life Russia interesting            |
|John and Peter are brothers. However they don't support each other that much.|John Peter brothers                |
|John and Peter are brothers. However they don't support each other that much.|However dont support much          |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |Lucas Nogal Dunbercker longer happy|
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.   

In [86]:
result.select('text', F.explode(result.clean_text.result).alias('clean_text')).toPandas()

,text,clean_text
0,Peter is a very good person.,Peter good person
1,My life in Russia is very interesting.,life Russia interesting
2,John and Peter are brothers. However they don'...,John Peter brothers
3,John and Peter are brothers. However they don'...,However dont support much
4,Lucas Nogal Dunbercker is no longer happy. He ...,Lucas Nogal Dunbercker longer happy
5,Lucas Nogal Dunbercker is no longer happy. He ...,good car though
6,Europe is very culture rich. There are huge ch...,Europe culture rich
7,Europe is very culture rich. There are huge ch...,huge churches
8,Europe is very culture rich. There are huge ch...,big houses


In [87]:
import pyspark.sql.functions as F

result.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").show(truncate = False)

+-----+---+-----------------------------------+--------+
|begin|end|result                             |sentence|
+-----+---+-----------------------------------+--------+
|0    |16 |Peter good person                  |0       |
|0    |22 |life Russia interesting            |0       |
|0    |18 |John Peter brothers                |0       |
|29   |53 |However dont support much          |1       |
|0    |34 |Lucas Nogal Dunbercker longer happy|0       |
|43   |57 |good car though                    |1       |
|0    |18 |Europe culture rich                |0       |
|29   |41 |huge churches                      |1       |
|54   |63 |big houses                         |2       |
+-----+---+-----------------------------------+--------+



In [88]:
# if we hadn't used Sentence Detector, this would be what we got. (tokenizer gets document instead of sentences column)

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

tokenassembler = TokenAssembler()\
    .setInputCols(["document", "cleanTokens"]) \
    .setOutputCol("clean_text")

nlpPipeline = Pipeline(stages=[documentAssembler,
                               tokenizer,
                               normalizer,
                               stopwords_cleaner,
                               tokenassembler])

result = nlpPipeline.fit(spark_df).transform(spark_df)
result.select('text', 'clean_text.result').show(truncate=False)

+-----------------------------------------------------------------------------+-----------------------------------------------------+
|text                                                                         |result                                               |
+-----------------------------------------------------------------------------+-----------------------------------------------------+
|Peter is a very good person.                                                 |[Peter good person]                                  |
|My life in Russia is very interesting.                                       |[life Russia interesting]                            |
|John and Peter are brothers. However they don't support each other that much.|[John Peter brothers However dont support much]      |
|Lucas Nogal Dunbercker is no longer happy. He has a good car though.         |[Lucas Nogal Dunbercker longer happy good car though]|
|Europe is very culture rich. There are huge churches! and big

In [89]:
result.withColumn(
    "tmp", 
    F.explode("clean_text")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").show(truncate = False)

+-----+---+---------------------------------------------------+--------+
|begin|end|result                                             |sentence|
+-----+---+---------------------------------------------------+--------+
|0    |16 |Peter good person                                  |0       |
|0    |22 |life Russia interesting                            |0       |
|0    |44 |John Peter brothers However dont support much      |0       |
|0    |50 |Lucas Nogal Dunbercker longer happy good car though|0       |
|0    |43 |Europe culture rich huge churches big houses       |0       |
+-----+---+---------------------------------------------------+--------+



**IMPORTANT NOTE:**

If you have some other steps & annotators in your pipeline that will need to use the tokens from cleaned text (assembled tokens), you will need to tokenize the processed text again as the original text is probably changed completely.

## Stemmer

Returns hard-stems out of words with the objective of retrieving the meaningful part of the word


In [90]:
stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

In [91]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               stemmer])

result = nlpPipeline.fit(spark_df).transform(spark_df)

result.show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|                                    text|                                document|                                   token|                                    stem|
+----------------------------------------+----------------------------------------+----------------------------------------+----------------------------------------+
|            Peter is a very good person.|[{document, 0, 27, Peter is a very go...|[{token, 0, 4, Peter, {sentence -> 0}...|[{token, 0, 4, peter, {sentence -> 0}...|
|  My life in Russia is very interesting.|[{document, 0, 37, My life in Russia ...|[{token, 0, 1, My, {sentence -> 0}, [...|[{token, 0, 1, my, {sentence -> 0}, [...|
|John and Peter are brothers. However ...|[{document, 0, 76, John and Peter are...|[{token, 0, 3, John, {sentence -> 0},...|[{token, 0, 3, john, {sentence -> 0},...|
|Luc

In [92]:
result.select('stem.result').show(truncate=False)

+-------------------------------------------------------------------------------------------+
|result                                                                                     |
+-------------------------------------------------------------------------------------------+
|[peter, i, a, veri, good, person, .]                                                       |
|[my, life, in, russia, i, veri, interest, .]                                               |
|[john, and, peter, ar, brother, ., howev, thei, don't, support, each, other, that, much, .]|
|[luca, nogal, dunberck, i, no, longer, happi, ., he, ha, a, good, car, though, .]          |
|[europ, i, veri, cultur, rich, ., there, ar, huge, church, !, and, big, hous, !]           |
+-------------------------------------------------------------------------------------------+



In [93]:
import pyspark.sql.functions as F

result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.stem.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("stem")).toPandas()

result_df.head(10)

,token,stem
0,Peter,peter
1,is,i
2,a,a
3,very,veri
4,good,good
5,person,person
6,.,.
7,My,my
8,life,life
9,in,in


## Lemmatizer

Retrieves lemmas out of words with the objective of returning a base dictionary word 

In [94]:
!wget -q https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

In [95]:
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

In [96]:
lemmatizer.extractParamMap()

{Param(parent='Lemmatizer_dc1f8ecdaf96', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='Lemmatizer_dc1f8ecdaf96', name='formCol', doc='Column that correspends to CoNLLU(formCol=) output'): 'form',
 Param(parent='Lemmatizer_dc1f8ecdaf96', name='lemmaCol', doc='Column that correspends to CoNLLU(lemmaCol=) output'): 'lemma',
 Param(parent='Lemmatizer_dc1f8ecdaf96', name='inputCols', doc='previous annotations columns, if renamed'): ['token'],
 Param(parent='Lemmatizer_dc1f8ecdaf96', name='outputCol', doc='output annotation column. can be left default.'): 'lemma',
 Param(parent='Lemmatizer_dc1f8ecdaf96', name='dictionary', doc="lemmatizer external dictionary. needs 'keyDelimiter' and 'valueDelimiter' in options for parsing target text"): JavaObject id=o2846}

In [97]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               stemmer,
                               lemmatizer])

result = nlpPipeline.fit(spark_df).transform(spark_df)
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[{document, 0, 27...|[{token, 0, 4, Pe...|[{token, 0, 4, pe...|[{token, 0, 4, Pe...|
|My life in Russia...|[{document, 0, 37...|[{token, 0, 1, My...|[{token, 0, 1, my...|[{token, 0, 1, My...|
|John and Peter ar...|[{document, 0, 76...|[{token, 0, 3, Jo...|[{token, 0, 3, jo...|[{token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[{document, 0, 67...|[{token, 0, 4, Lu...|[{token, 0, 4, lu...|[{token, 0, 4, Lu...|
|Europe is very cu...|[{document, 0, 68...|[{token, 0, 5, Eu...|[{token, 0, 5, eu...|[{token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [98]:
result.select('lemma.result').show(truncate=False)

+---------------------------------------------------------------------------------------------+
|result                                                                                       |
+---------------------------------------------------------------------------------------------+
|[Peter, be, a, very, good, person, .]                                                        |
|[My, life, in, Russia, be, very, interest, .]                                                |
|[John, and, Peter, be, brother, ., However, they, don't, support, each, other, that, much, .]|
|[Lucas, Nogal, Dunbercker, be, no, long, happy, ., He, have, a, good, car, though, .]        |
|[Europe, be, very, culture, rich, ., There, be, huge, church, !, and, big, house, !]         |
+---------------------------------------------------------------------------------------------+



In [99]:
result_df = result.select(F.explode(F.arrays_zip(result.token.result, 
                                                 result.stem.result, 
                                                 result.lemma.result)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("token"),
                          F.expr("cols['1']").alias("stem"),
                          F.expr("cols['2']").alias("lemma")).toPandas()

result_df.head(10)

,token,stem,lemma
0,Peter,peter,Peter
1,is,i,be
2,a,a,a
3,very,veri,very
4,good,good,good
5,person,person,person
6,.,.,.
7,My,my,My
8,life,life,life
9,in,in,in


## NGram Generator

NGramGenerator annotator takes as input a sequence of strings (e.g. the output of a `Tokenizer`, `Normalizer`, `Stemmer`, `Lemmatizer`, and `StopWordsCleaner`). 

The parameter n is used to determine the number of terms in each n-gram. The output will consist of a sequence of n-grams where each n-gram is represented by a space-delimited string of n consecutive words with annotatorType `CHUNK` same as the Chunker annotator.

Functions:

`setN:` number elements per n-gram (>=1)

`setEnableCumulative:` whether to calculate just the actual n-grams or all n-grams from 1 through n

`setDelimiter:` Glue character used to join the tokens

In [100]:
ngrams_cum = NGramGenerator() \
                .setInputCols(["token"]) \
                .setOutputCol("ngrams") \
                .setN(3) \
                .setEnableCumulative(True)\
                .setDelimiter("_") # Default is space
    
# .setN(3) means, take bigrams and trigrams.

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               tokenizer,
                               ngrams_cum])

result = nlpPipeline.fit(spark_df).transform(spark_df)
result.select('ngrams.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                    [Peter, is, a, very, good, person, ., Peter_is, is_a, a_very, very_good, good_person, person_., Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
|[My, life, in, Russia, is, very, interesting, ., My_life, life_in, in_Russia, Russia_is, is_very, very_interesting, interesting_., My_life_in, life_in_Russia, in_Russia_is, Russia_is_very

In [101]:
ngrams_nonCum = NGramGenerator() \
                  .setInputCols(["token"]) \
                  .setOutputCol("ngrams_v2") \
                  .setN(3) \
                  .setEnableCumulative(False)\
                  .setDelimiter("_") # Default is space
    
ngrams_nonCum.transform(result).select('ngrams_v2.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                   [Peter_is_a, is_a_very, a_very_good, very_good_person, good_person_.]|
|                                                                                                     [My_life_in, life_in_Russia, in_Russia_is, Russia_is_very, is_very_interesting, very_i

## TextMatcher

Annotator to match entire phrases (by token) provided in a file against a Document

Functions:

`setEntities(path, format, options)`: Provides a file with phrases to match. Default: Looks up path in configuration.

`path`: a path to a file that contains the entities in the specified format.

`readAs`: the format of the file, can be one of {ReadAs.LINE_BY_LINE, ReadAs.SPARK_DATASET}. Defaults to LINE_BY_LINE.

`options`: a map of additional parameters. Defaults to {“format”: “text”}.

`entityValue` : Value for the entity metadata field to indicate which chunk comes from which textMatcher when there are multiple textMatchers. 

`mergeOverlapping` : whether to merge overlapping matched chunks. Defaults false

`caseSensitive` : whether to match regardless of case. Defaults true


In [102]:
entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("matched_entities")\

entity_extractor.extractParamMap()

{Param(parent='TextMatcher_0abb8d79cdf8', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='TextMatcher_0abb8d79cdf8', name='inputCols', doc='previous annotations columns, if renamed'): ['document',
  'token'],
 Param(parent='TextMatcher_0abb8d79cdf8', name='caseSensitive', doc='whether to match regardless of case. Defaults true'): True,
 Param(parent='TextMatcher_0abb8d79cdf8', name='mergeOverlapping', doc='whether to merge overlapping matched chunks. Defaults false'): False,
 Param(parent='TextMatcher_0abb8d79cdf8', name='outputCol', doc='output annotation column. can be left default.'): 'matched_entities'}

In [103]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv

news_df = spark.read \
            .option("header", True) \
            .csv("news_category_train.csv")


In [104]:
news_df.show(5, truncate=50)

+--------+--------------------------------------------------+
|category|                                       description|
+--------+--------------------------------------------------+
|Business| Short sellers, Wall Street's dwindling band of...|
|Business| Private investment firm Carlyle Group, which h...|
|Business| Soaring crude prices plus worries about the ec...|
|Business| Authorities have halted oil export flows from ...|
|Business| Tearaway world oil prices, toppling records an...|
+--------+--------------------------------------------------+
only showing top 5 rows



In [105]:
# write the target entities to txt file 

entities = ['Wall Street', 'USD', 'stock', 'NYSE']
with open ('financial_entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')


entities = ['soccer', 'world cup', 'Messi', 'FC Barcelona']
with open ('sport_entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')


In [106]:
documentAssembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

financial_entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("financial_entities")\
    .setEntities("financial_entities.txt")\
    .setCaseSensitive(False)\
    .setEntityValue('financial_entity')

sport_entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("sport_entities")\
    .setEntities("sport_entities.txt")\
    .setCaseSensitive(False)\
    .setEntityValue('sport_entity')

nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        tokenizer,
        financial_entity_extractor,
        sport_entity_extractor
        ])

result = nlpPipeline.fit(news_df).transform(news_df)

In [107]:
result.select('financial_entities.result','sport_entities.result').take(3)

[Row(result=[], result=[]),
 Row(result=[], result=[]),
 Row(result=['stock'], result=[])]

In [108]:
result.select('description','financial_entities.result','sport_entities.result')\
      .toDF('text','financial_matches','sport_matches').filter((F.size('financial_matches')>1) | (F.size('sport_matches')>1))\
      .show(truncate=70)

+----------------------------------------------------------------------+----------------------------------+-------------------+
|                                                                  text|                 financial_matches|      sport_matches|
+----------------------------------------------------------------------+----------------------------------+-------------------+
|"Company launched the biggest electronic auction of stock in Wall S...|              [stock, Wall Street]|                 []|
|Google, Inc. significantly cut the expected share price for its ini...|                    [stock, stock]|                 []|
|Google, Inc. significantly cut the expected share price this mornin...|                    [stock, stock]|                 []|
| Shares of Air Canada  (AC.TO) fell by more than half on Wednesday,...|                    [Stock, stock]|                 []|
|Stock prices are lower in moderate trading. The Dow Jones Industria...|                    [Stock, Stoc

In [109]:
result_df = result.select(F.explode(F.arrays_zip(result.financial_entities.result, 
                                                 result.financial_entities.begin, 
                                                 result.financial_entities.end)).alias("cols")) \
                  .select(F.expr("cols['0']").alias("clinical_entities"),
                          F.expr("cols['1']").alias("begin"),
                          F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,clinical_entities,begin,end
0,stock,112,116
1,stock,114,118
2,stock,45,49
3,stock,126,130
4,stock,188,192
5,stock,52,56
6,Wall Street,61,71
7,stock,70,74
8,stock,143,147
9,stock,294,298


## RegexMatcher

In [110]:
! wget -q	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv

pubMedDF = spark.read\
              .option("header", "true")\
              .csv("./pubmed-sample.csv")\
              .filter("AB IS NOT null")\
              .withColumnRenamed("AB", "text")\
              .drop("TI")

pubMedDF.show(truncate=50)

+--------------------------------------------------+
|                                              text|
+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
|BACKGROUND: At present, it is one of the most i...|
|OBJECTIVE: To investigate the relationship betw...|
|Combined EEG/fMRI recording has been used to lo...|
|Kohlschutter syndrome is a rare neurodegenerati...|
|Statistical analysis of neuroimages is commonly...|
|The synthetic DOX-LNA conjugate was characteriz...|
|Our objective was to compare three different me...|
|We conducted a phase II study to assess the eff...|
|"Monomeric sarcosine oxidase (MSOX) is a flavoe...|
|We presented the tachinid fly Exorista japonica...|
|The literature dealing with the water conductin...|
|A novel approach to synthesize chitosan-O-isopr...|
|An HPLC-ESI-MS-MS method has been developed for...|
|The localizing and lateralizing values of eye a...|
|OBJECTIVE: To evaluate the effectiveness and 

**setExternalRules**

In [111]:
rules = '''
renal\s\w+, started with 'renal'
cardiac\s\w+, started with 'cardiac'
\w*ly\b, ending with 'ly'
\S*\d+\S*, match any word that contains numbers
(\d+).?(\d*)\s*(mg|ml|g), match medication metrics
'''

with open('regex_rules.txt', 'w') as f:
    
    f.write(rules)
    

In [112]:
RegexMatcher().extractParamMap()

{Param(parent='RegexMatcher_7bc859cc7b3b', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='RegexMatcher_7bc859cc7b3b', name='strategy', doc='MATCH_FIRST|MATCH_ALL|MATCH_COMPLETE'): 'MATCH_ALL'}

In [113]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

regex_matcher = RegexMatcher()\
    .setInputCols('document')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./regex_rules.txt', delimiter=',')
    

nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        regex_matcher
        ])

match_df = nlpPipeline.fit(pubMedDF).transform(pubMedDF)
match_df.select('regex_matches.result').take(3)

[Row(result=['inwardly', 'family', 'spansapproximately', 'byapproximately', 'approximately', 'respectively', 'poly', 'KCNJ9', '3.3,', 'GIRK3)', 'KCNJ9', '1q21-23', '7.6', '2.2', '2.6', 'identified14', 'aVal366Ala', '8', 'KCNJ9', 'KCNJ9', '9 g']),
 Row(result=['previously', 'previously', 'intravenously', 'previously', '25', 'mg/m(2)', '1', '8', 'a3', '50', '20.0%', '(10', '50;', '95%', 'interval,10.0-33.7%).', '58.0%', '[10', '18', '50].', '(50%', '115.0', '17.3%', '52).', '25 mg']),
 Row(result=['renal failure', 'cardiac surgery', 'cardiac surgery', 'cardiac surgical', 'early', 'statistically', 'analy', '1995', '2005', '=9796).', '2.9', '11years).', '11.3%', '1105),', '7.2%', '30%', '0.0001),', '1.55,95%', '1.42-1.70,', '0.0001).'])]

In [114]:
match_df.select('text','regex_matches.result')\
        .toDF('text','matches').filter(F.size('matches')>1)\
        .show(10, truncate=70)

+----------------------------------------------------------------------+----------------------------------------------------------------------+
|                                                                  text|                                                               matches|
+----------------------------------------------------------------------+----------------------------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activ...|[inwardly, family, spansapproximately, byapproximately, approximate...|
|BACKGROUND: At present, it is one of the most important issues for ...|[previously, previously, intravenously, previously, 25, mg/m(2), 1,...|
|OBJECTIVE: To investigate the relationship between preoperative atr...|[renal failure, cardiac surgery, cardiac surgery, cardiac surgical,...|
|Combined EEG/fMRI recording has been used to localize the generator...|[normally, significantly, effectively, analy, only, considerably

**setRules**

In [115]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

regexmatcher = RegexMatcher()\
    .setRules(["\\d{4}\\/\\d\\d\\/\\d\\d,date", "\\d{2}\\/\\d\\d\\/\\d\\d,short_date"])\
    .setDelimiter(",") \
    .setInputCols('document')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")
    

nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        regexmatcher
        ])

match_df2 = nlpPipeline.fit(pubMedDF).transform(pubMedDF)

In [116]:
match_df2.select('text','regex_matches.result')\
        .toDF('text','matches').filter(F.size('matches')>1)\
        .show(10, truncate=70)

+----------------------------------------------------------------------+--------------------+
|                                                                  text|             matches|
+----------------------------------------------------------------------+--------------------+
|BACKGROUND CONTEXT: In the last decade, the number of patients unde...|[64/63/65, 27/24/27]|
+----------------------------------------------------------------------+--------------------+



## MultiDateMatcher

Extract exact & normalize dates from relative date-time phrases. The default anchor date will be the date the code is run.

In [117]:
MultiDateMatcher().extractParamMap()

{Param(parent='MultiDateMatcher_ec7f2e2e79e4', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='MultiDateMatcher_ec7f2e2e79e4', name='inputFormats', doc='input formats list of patterns to match'): [''],
 Param(parent='MultiDateMatcher_ec7f2e2e79e4', name='outputFormat', doc='desired output format for dates extracted'): 'yyyy/MM/dd',
 Param(parent='MultiDateMatcher_ec7f2e2e79e4', name='readMonthFirst', doc='Whether to parse july 07/05/2015 or as 05/07/2015'): True,
 Param(parent='MultiDateMatcher_ec7f2e2e79e4', name='defaultDayWhenMissing', doc='which day to set when it is missing from parsed input'): 1}

In [118]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

date_matcher = MultiDateMatcher() \
    .setInputCols('document') \
    .setOutputCol("date")\
    .setOutputFormat("yyyy/MM/dd")\
    .setSourceLanguage("en")


date_pipeline = PipelineModel(
    stages=[
        documentAssembler, 
        date_matcher
        ])

sample_df = spark.createDataFrame([['I saw him yesterday and he told me that he will visit us next week']]).toDF("text")

result = date_pipeline.transform(sample_df)
result.select('date.result').show(truncate=False)

+------------------------+
|result                  |
+------------------------+
|[2023/01/22, 2023/01/14]|
+------------------------+



Let's set the Input Format and Output Format to specific format

In [119]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

date_matcher = MultiDateMatcher() \
    .setInputCols('document') \
    .setOutputCol("date")\
    .setInputFormats(["dd/MM/yyyy"])\
    .setOutputFormat("yyyy/MM/dd")\
    .setSourceLanguage("en")

date_pipeline = PipelineModel(
    stages=[
        documentAssembler, 
        date_matcher
        ])

sample_df = spark.createDataFrame([["the last payment date of this invoice is 21/05/2022"]]).toDF("text")

result = date_pipeline.transform(sample_df)
result.select('date.result').show(truncate=False)

+------------+
|result      |
+------------+
|[2022/05/21]|
+------------+



## Text Cleaning with UDF

In [120]:
text = '<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>'

text_df = spark.createDataFrame([[text]]).toDF("text")

import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType

clean_text = lambda s: re.sub(r'<[^>]*>', '', s)

text_df.withColumn('cleaned', udf(clean_text, StringType())('text')).select('text','cleaned').show(truncate= False)

+----------------------------------------------------------------------------------------------+-----------------------+
|text                                                                                          |cleaned                |
+----------------------------------------------------------------------------------------------+-----------------------+
|<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>|Have a great birth day!|
+----------------------------------------------------------------------------------------------+-----------------------+



In [121]:
find_not_alnum_count = lambda s: len([i for i in s if not i.isalnum() and i!=' '])

find_not_alnum_count("it's your birth day!")

2

In [122]:
text = '<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>'

find_not_alnum_count(text)

23

In [123]:
text_df.withColumn('cleaned', udf(find_not_alnum_count, IntegerType())('text')).select('text','cleaned').show(truncate= False)

+----------------------------------------------------------------------------------------------+-------+
|text                                                                                          |cleaned|
+----------------------------------------------------------------------------------------------+-------+
|<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>|23     |
+----------------------------------------------------------------------------------------------+-------+



## Finisher

***Finisher:*** Once we have our NLP pipeline ready to go, we might want to use our annotation results somewhere else where it is easy to use. The Finisher outputs annotation(s) values into a string.

If we just want the desired output column in the final dataframe, we can use Finisher to drop previous stages in the final output and get the `result` from the process.

This is very handy when you want to use the output from Spark NLP annotator as an input to another Spark ML transformer.


In [125]:
finisher = Finisher() \
    .setInputCols(["regex_matches"]) \
    .setIncludeMetadata(False) # set to False to remove metadata

nlpPipeline = Pipeline(stages=[documentAssembler, 
                               regex_matcher,
                               finisher])

match_df = nlpPipeline.fit(pubMedDF).transform(pubMedDF)
match_df.take(1)

[Row(text='The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family. Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population. The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively. We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair (bp) insertion/deletion. Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle. The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.', finished_regex_matches=['inwa

In [126]:
match_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- finished_regex_matches: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [127]:
match_df.filter(F.size('finished_regex_matches')>2).show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|[inwardly, family, spansapproximately, byapprox...|
|BACKGROUND: At present, it is one of the most i...|[previously, previously, intravenously, previou...|
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Combined EEG/fMRI recording has been used to lo...|[normally, significantly, effectively, analy, o...|
|Statistical analysis of neuroimages is commonly...|[analy, commonly, overly, normally, thatsuccess...|
|Our objective was to compare three different me...|[daily, only, Conversely, Hourly, hourly, Hourl...|
|We conducted a phase II study to assess the eff...|[analy, resp

## LightPipeline

https://medium.com/spark-nlp/spark-nlp-101-lightpipeline-a544e93f20f1

LightPipelines are Spark NLP specific Pipelines, equivalent to Spark ML Pipeline, but meant to deal with smaller amounts of data. They’re useful working with small datasets, debugging results, or when running either training or prediction from an API that serves one-off requests.

Spark NLP LightPipelines are Spark ML pipelines converted into a single machine but the multi-threaded task, becoming more than 10x times faster for smaller amounts of data (small is relative, but 50k sentences are roughly a good maximum). To use them, we simply plug in a trained (fitted) pipeline and then annotate a plain text. We don't even need to convert the input text to DataFrame in order to feed it into a pipeline that's accepting DataFrame as an input in the first place. This feature would be quite useful when it comes to getting a prediction for a few lines of text from a trained ML model.

 **It is nearly 10x faster than using Spark ML Pipeline**

`LightPipeline(someTrainedPipeline).annotate(someStringOrArray)`

In [128]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(
    stages=[
        documentAssembler, 
        tokenizer,
        stemmer,
        lemmatizer
        ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)
nlpPipeline.fit(empty_df).transform(spark_df).show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a very g...|[{document, 0, 27...|[{token, 0, 4, Pe...|[{token, 0, 4, pe...|[{token, 0, 4, Pe...|
|My life in Russia...|[{document, 0, 37...|[{token, 0, 1, My...|[{token, 0, 1, my...|[{token, 0, 1, My...|
|John and Peter ar...|[{document, 0, 76...|[{token, 0, 3, Jo...|[{token, 0, 3, jo...|[{token, 0, 3, Jo...|
|Lucas Nogal Dunbe...|[{document, 0, 67...|[{token, 0, 4, Lu...|[{token, 0, 4, lu...|[{token, 0, 4, Lu...|
|Europe is very cu...|[{document, 0, 68...|[{token, 0, 5, Eu...|[{token, 0, 5, eu...|[{token, 0, 5, Eu...|
+--------------------+--------------------+--------------------+--------------------+--------------------+



In [129]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)
light_result = light_model.annotate("John and Peter are brothers. However they don't support each other that much.")

In [130]:
light_result.keys()

dict_keys(['document', 'token', 'stem', 'lemma'])

In [131]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma']))

[('John', 'john', 'John'),
 ('and', 'and', 'and'),
 ('Peter', 'peter', 'Peter'),
 ('are', 'ar', 'be'),
 ('brothers', 'brother', 'brother'),
 ('.', '.', '.'),
 ('However', 'howev', 'However'),
 ('they', 'thei', 'they'),
 ("don't", "don't", "don't"),
 ('support', 'support', 'support'),
 ('each', 'each', 'each'),
 ('other', 'other', 'other'),
 ('that', 'that', 'that'),
 ('much', 'much', 'much'),
 ('.', '.', '.')]

In [132]:
light_result = light_model.fullAnnotate("John and Peter are brothers. However they don't support each other that much.")

In [ ]:
light_result

In [133]:
text_list= ["How did serfdom develop in and then leave Russia ?",
"There will be some exciting breakthroughs in NLP this year."]

light_model.annotate(text_list)

[{'document': ['How did serfdom develop in and then leave Russia ?'],
  'token': ['How',
   'did',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leave',
   'Russia',
   '?'],
  'stem': ['how',
   'did',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leav',
   'russia',
   '?'],
  'lemma': ['How',
   'do',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leave',
   'Russia',
   '?']},
 {'document': ['There will be some exciting breakthroughs in NLP this year.'],
  'token': ['There',
   'will',
   'be',
   'some',
   'exciting',
   'breakthroughs',
   'in',
   'NLP',
   'this',
   'year',
   '.'],
  'stem': ['there',
   'will',
   'be',
   'some',
   'excit',
   'breakthrough',
   'in',
   'nlp',
   'thi',
   'year',
   '.'],
  'lemma': ['There',
   'will',
   'be',
   'some',
   'exciting',
   'breakthrough',
   'in',
   'NLP',
   'this',
   'year',
   '.']}]

**important note:** When you use Finisher in your pipeline, regardless of setting `cleanAnnotations` to False or True, LigtPipeline will only return the finished columns.